In [24]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [25]:
!python -V

Python 3.9.19


In [26]:
import pickle
import pandas as pd

In [27]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/ubuntu/anaconda3/envs/mlflow_env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/anaconda3/envs/mlflow_env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [28]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [29]:
year = 2023
month = 3

In [30]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [31]:
# Create ride_id column
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [32]:
# Prepare the data for prediction
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [33]:
std_pred_duration = y_pred.std()
print(f'Standard deviation of predicted duration: {std_pred_duration}')

Standard deviation of predicted duration: 6.247488852238703


In [34]:
# Create result dataframe
df_result = pd.DataFrame({'ride_id': df['ride_id'], 'predicted_duration': y_pred})

In [35]:
#Save result dataframe as parquet file
output_file = 'result_2023-03.parquet'
df_result.to_parquet(output_file, engine='pyarrow', compression=None, index=False)


In [36]:
!ls -l

total 66780
-rw-rw-r-- 1 ubuntu ubuntu      221 Jun 17 04:14 Pipfile
-rw-r--r-- 1 ubuntu ubuntu    18609 Jun 17 04:14 Pipfile.lock
-rw-rw-r-- 1 ubuntu ubuntu    17376 Jun 11 03:35 model.bin
-rw-rw-r-- 1 ubuntu ubuntu 68322651 Jun 17 04:28 result_2023-03.parquet
-rw-rw-r-- 1 ubuntu ubuntu     5924 Jun 17 04:27 starter.ipynb
-rw-rw-r-- 1 ubuntu ubuntu     1694 Jun 17 04:22 starter.py
